# Data Exploration: Module Style

In [13]:
import sys
sys.path.append("../")

import numpy as np
import math
from cosmic_rAI import data_prep

## Load and Prep Data

In [14]:
mat1 = np.load('../data/sim_12360_00.npy').item()  # protons
mat2 = np.load('../data/sim_12362_00.npy').item()  # iron

In [15]:
mat1.keys()

dict_keys(['Charges', 'Energy', 'File_info', 'dir_reco', 'core_MC', 'Gain', 'core_reco', 'Position', 'dir_MC', 'Fit_status', 'Composition'])

### Cleanup, Gain Differentiate

In [16]:
# combine expanded attributes, like MC and Reco
data_prep.cleanup((mat1, mat2))

In [17]:
# split into hgain and lgain datasets
hgain, lgain, all_sensors = data_prep.gain_differentiate(mat1)

In [18]:
# Construct dictionary that goes name->column index
hgain_indices = data_prep.get_index_dict(hgain)
lgain_indices = data_prep.get_index_dict(lgain)
all_indices = data_prep.get_index_dict(all_sensors)

In [19]:
# Make matrix, using indices created above
hgain_events = data_prep.gen_matrix(mat1, mat2, hgain_indices)
lgain_events = data_prep.gen_matrix(mat1, mat2, lgain_indices)

In [20]:
hgain_events.shape

(31620, 163)

### Split high-gain into training/test set

In [34]:
# Split into testing and training sets
trainset, testset = data_prep.split_train(hgain_events)
np.save('../data_processed/small_train_high_low.npy', trainset)
np.save('../data_processed/small_test_high_low.npy', testset)

In [35]:
# Add azimuth, zenith features to dataset

all_indices_dir = data_prep.get_index_dict(all_sensors, direction_data=True)
all_events_direction = data_prep.gen_matrix(mat1, mat2, all_indices_dir, direction=True)
trainset, testset = data_prep.split_train(all_events_direction)

np.save('../data_processed/direction_train.npy',trainset)
np.save('../data_processed/direction_test.npy', testset)